If you are using VSCode, simply clone the repository in a container volume. Otherwise, you'll need standard Anaconda packages as well as [CVXPY](https://www.cvxpy.org/install/).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

Helper function for later

In [2]:
def get_dispatch(market, total):
    dispatch = np.minimum(market.Capacity.cumsum(), total)
    dispatch = dispatch.diff().fillna(dispatch)
    return dispatch

Load and process portfolio data

In [3]:
portfolios = pd.read_excel('ESG Portfolios.xlsx')
portfolios.columns = portfolios.iloc[0:3].bfill().iloc[0]
portfolios = portfolios.iloc[3:]
portfolios = portfolios[portfolios['UNIT NAME'] != 'Totals']
portfolios.dropna(axis='index', how='all', inplace=True)
portfolios.dropna(axis='columns', how='all', inplace=True)
portfolios['portfolio'] = (portfolios.isna().any(axis='columns') * portfolios['UNIT NAME']).replace('', np.NaN).ffill()
portfolios.dropna(axis='index', how='any', inplace=True)
portfolios.set_index(['Location', 'portfolio', 'UNIT NAME'], inplace=True)
portfolios.sort_index(axis='index',level=['Location', 'portfolio'], inplace=True)

Load and process profile data

In [4]:
profiles = pd.read_excel('ESG Demand Year 2.xlsx', header=2, index_col=[1,0,2])
profiles.dropna(axis='index', how='any', inplace=True)
profiles.sort_index(inplace=True)

For now, we'll just simulate the first hour of the first day. Edit this line to choose a different hour and/or day.

In [5]:
hour = profiles.loc[(['North', 'South'], 1, 1)].droplevel(['Day', 'Hour'])

This cell adds a column to the ``portfolio`` dataframe containing the dispatched generation, computed according to the bids. See the comments in the cell for ways to modify the simulation.

In [6]:
# Set the bids. 
# The name of the column must remain 'Bid'.
# To test a different strategy than marginal cost, edit the right-hand side.
portfolios['Bid'] = portfolios['Total Marginal Cost']

portfolios = portfolios.groupby('Location', group_keys=False).apply(lambda df: df.sort_values('Bid'))

q_g = cp.Variable(2)
q_d = cp.Variable(2)
q_t = cp.Variable()

objective = cp.Minimize(
    cp.sum(
        [
            cp.max((market.Capacity*market.Bid).cumsum().to_numpy() + cp.multiply(market.Bid,(q_g[i] - market.Capacity.cumsum())))
            - (q_d[i]**2/2 - hour.loc[location, 'Load Intercept']*q_d[i])/hour.loc[location, 'Load Slope']
            for i, (location, market) in enumerate(portfolios.groupby('Location'))
        ]
    )
)

constraints = [
    q_g == q_d + [1, -1]*q_t,
    q_d >= 0,
    q_g >= 0,
    q_g <= portfolios.Capacity.groupby('Location').sum(),
    # Edit the right-hand side of this constraint to change the transmission capacity
    cp.abs(q_t) <= 10000
]

cp.Problem(
    objective, constraints
).solve(solver='CLARABEL')

q_g = dict(zip(portfolios.index.unique('Location'), q_g.value))
q_d = dict(zip(portfolios.index.unique('Location'), q_d.value))

portfolios['Dispatch'] = portfolios.groupby('Location', group_keys=False).apply(
    lambda market: get_dispatch(market, q_g[market.name])
)

Check the results. To check profit, compute the price by evaluating the marginal utility function for each region at the consumption in that region. The consumption by region is contained in the variable ``q_d``.

In [7]:
portfolios

0                                              Fuel Capacity  Heat Rate  \
Location portfolio    UNIT NAME                                           
North    Fossil Light HELMS                   Hydro      800          0   
                      DIABLO CANYON 1       Nuclear     1000          1   
         Bay Views    MOSS LANDING 6            Gas      750   6.901235   
                      MOSS LANDING 7            Gas      750   6.901235   
                      MORRO BAY 3&4             Gas      665   8.024691   
         East Bay     PITTSBURGH 5&6            Gas      650   8.024691   
         Bay Views    MORRO BAY 1&2             Gas      335   8.506173   
         East Bay     CONTRA COSTA 6&7          Gas      700   8.666667   
                      PITTSBURGH 1-4            Gas      650   8.987654   
         Fossil Light HUMBOLDT                  Gas      150  10.432099   
                      HUNTERS POINT 1&2         Gas      150  10.592593   
         East Bay     CONTRA COSTA 4&5          Gas      150  12.839506   
                      PITTSBURGH 7              Gas      700  13.160494   
         Bay Views    OAKLAND                   Gas      150  13.481481   
         East Bay     POTRERO HILL              Gas      150  15.407407   
         Fossil Light HUNTERS POINT 4           Gas      250  16.530864   
South    Old Timers   BIG CREEK               Hydro     1000          0   
                      MOHAVE 1                 Coal      750         10   
                      MOHAVE 2                 Coal      750         10   
         Big Coal     FOUR CORNERS             Coal     1900  11.666667   
         Beachfront   ORMOND BEACH 1            Gas      700   8.345679   
                      ORMOND BEACH 2            Gas      700   8.345679   
                      MANDALAY 1&2              Gas      300   8.345679   
         Big Coal     HUNTINGTON BEACH 1&2      Gas      300   8.666667   
         Big Gas      EL SEGUNDO 3&4            Gas      650    8.82716   
                      ENCINA                    Gas      950   9.148148   
         Big Coal     REDONDO 5&6               Gas      350   8.987654   
                      REDONDO 7&8               Gas      950   8.987654   
         Beachfront   COOLWATER                 Gas      650   9.308642   
                      ETIWANDA 1-4              Gas      850   9.148148   
         Big Gas      SOUTH BAY                 Gas      700    9.62963   
                      EL SEGUNDO 1&2            Gas      400    9.62963   
         Old Timers   HIGHGROVE                 Gas      150   10.91358   
         Beachfront   MANDALAY 3                Gas      150  11.234568   
         Big Gas      LONG BEACH                Gas      550  11.555556   
         Old Timers   SAN BERNADINO             Gas      100  11.876543   
         Beachfront   ETIWANDA 5                Gas      150  13.641975   
         Big Gas      NORTH ISLAND              Gas      150  14.444444   
         Big Coal     HUNTINGTON BEACH 5        Gas      150  14.444444   
                      ALAMITOS 7                Gas      250  16.049383   
         Beachfront   ELLWOOD                   Gas      300  16.691358   
         Big Gas      KEARNY                    Gas      200  19.901235   

0                                          Fuel Price  Fuel Cost O&M/MWh  \
Location portfolio    UNIT NAME                                            
North    Fossil Light HELMS                         0          0     0.5   
                      DIABLO CANYON 1             7.5        7.5       4   
         Bay Views    MOSS LANDING 6              4.5  31.055556     1.5   
                      MOSS LANDING 7              4.5  31.055556     1.5   
                      MORRO BAY 3&4               4.5  36.111111     0.5   
         East Bay     PITTSBURGH 5&6              4.5  36.111111     0.5   
         Bay Views    MORRO BAY 1&2               4.5  38.277778     0.5   
         East B